#### CS20M059 Shibobrota Das | CS20M007 Abhishek Kumar

## Setup

In [44]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
import pandas as pd
import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN, SimpleRNNCell, LSTMCell, GRUCell
from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
import os
import time
from sklearn.utils import shuffle

print("Using numpy:",np.__version__)
print("Using tensorflow:",tf.__version__)
print("Using tensorflow Addons:",tfa.__version__)
print("Using keras:",keras.__version__)
print("Using pandas:",pd.__version__)

Using numpy: 1.19.5
Using tensorflow: 2.4.1
Using tensorflow Addons: 0.12.1
Using keras: 2.4.0
Using pandas: 1.2.3


#### Load Data

In [11]:
val_df = pd.read_csv("./lexicons/hi.translit.sampled.dev.tsv", sep='\t', header=None)
train_df = pd.read_csv("./lexicons/hi.translit.sampled.train.tsv", sep='\t', header=None)
test_df = pd.read_csv("./lexicons/hi.translit.sampled.test.tsv", sep='\t', header=None)
print("Data Loaded to Dataframes!")

Data Loaded to Dataframes!


#### Dataset Samples

In [3]:
val_df.sample(n=3)

,0,1,2
1919,दुगुने,dugune,3
3704,वैष्णो,vaishno,3
4243,हब्शी,habshi,2


## Preparing Dataset

In [4]:
sos = "@"
eos = "#"

In [12]:
class LexDataset:
    def __init__(self, input_tensor, target_tensor, batch_size):
        self.input_tensor = input_tensor
        self.target_tensor = target_tensor
        self.batch = tf.data.Dataset.from_tensor_slices((self.input_tensor, self.target_tensor)).shuffle(len(self.input_tensor)).batch(batch_size, drop_remainder=True)

In [13]:
class TransliterationDatatset:
    def __init__(self, df_list, problem_type = "en-hi", batch_size = 32):
        self.problem_type = problem_type
        self.input_tokenizer = None
        self.target_tokenizer = None
        self.train = None
        self.val = None
        self.test = None
        self.batch_size = batch_size
        # Load Data
        self.load_dataset(df_list)
        
    def preprocess_word(self, w):
        return sos + str(w) + eos
    
    def create_dataset(self, data_frame):
        input_words = []
        target_words = []
        # Shuffle the data_frame before creating dataset
        df_shuffled = shuffle(data_frame)
        for x, y in zip(df_shuffled[1], df_shuffled[0]):
            input_words.append(self.preprocess_word(x))
            target_words.append(self.preprocess_word(y))
        return (input_words, target_words)
    
    def load_dataset(self, df_list):
        # df_list should have train -> val -> test in sequence
        
        self.input_tokenizer = Tokenizer(num_words = None, char_level = True)
        self.target_tokenizer = Tokenizer(num_words = None, char_level = True)
        
        ds_list = []
        
        for df in df_list:
            # Get the words list
            (input_words, target_words) = self.create_dataset(df)
            # Fit on the set of words
            self.input_tokenizer.fit_on_texts(input_words)
            self.target_tokenizer.fit_on_texts(target_words)
            ds_list.append((input_words, target_words))
                    
        self.target_tokenizer.index_word.update({0:" "})
        self.input_tokenizer.index_word.update({0:" "})
        
        for i, (input_words, target_words) in enumerate(ds_list):
            
            input_tensor = self.input_tokenizer.texts_to_sequences(input_words)
            input_tensor = pad_sequences(input_tensor, padding='post')
            
            target_tensor = self.target_tokenizer.texts_to_sequences(target_words)
            target_tensor = pad_sequences(target_tensor, padding='post')
            
            if i == 0:
                self.train = LexDataset(input_tensor, target_tensor, self.batch_size)
            elif i == 1:
                self.val = LexDataset(input_tensor, target_tensor, self.batch_size)
            else:
                self.test = LexDataset(input_tensor, target_tensor, self.batch_size)

In [14]:
dataset = TransliterationDatatset([train_df, val_df, test_df])

#### Training Data

In [15]:
# Training data
dataset.train.input_tensor.shape, dataset.train.target_tensor.shape

((44204, 22), (44204, 21))

#### Validation Data

In [16]:
# Validation data
dataset.val.input_tensor.shape, dataset.val.target_tensor.shape

((4358, 20), (4358, 16))

#### Test Data

In [17]:
# Test data
dataset.test.input_tensor.shape, dataset.test.target_tensor.shape

((4502, 18), (4502, 17))

#### Number of Tokens

In [18]:
# Number of tokens
num_encoder_tokens = len(dataset.input_tokenizer.index_word)+1
num_decoder_tokens = len(dataset.target_tokenizer.index_word)+1
num_encoder_tokens, num_decoder_tokens

(30, 67)

#### Maximum Sequence Lengths

In [19]:
# max seq length
max_encoder_seq_length = np.max([dataset.train.input_tensor.shape[1], dataset.val.input_tensor.shape[1], dataset.test.input_tensor.shape[1]])
max_decoder_seq_length = np.max([dataset.train.target_tensor.shape[1], dataset.val.target_tensor.shape[1], dataset.test.target_tensor.shape[1]])
max_encoder_seq_length, max_decoder_seq_length

(22, 21)

#### Example batch - dataset

In [20]:
example_input_batch, example_target_batch = next(iter(dataset.train.batch))
example_input_batch.shape, example_target_batch.shape

(TensorShape([32, 22]), TensorShape([32, 21]))

#### Required Parameters

In [21]:
embedding_dim = 16
units = 128
steps_per_epoch = np.shape(dataset.train.input_tensor)[0]//dataset.batch_size
steps_per_epoch

1381

## Encoder

In [22]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, encoder_units, batch_size, layer_type = "SimpleRNN"):
        super(Encoder, self).__init__()
        
        self.batch_size = batch_size
        self.encoder_units = encoder_units
        self.layer_type = layer_type
        self.embedding = Embedding(vocab_size, embedding_dim)
        
        ##-------- RNN layer in Encoder ------- ##
        if self.layer_type == "LSTM":
            self.layer = LSTM(self.encoder_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        elif self.layer_type == "GRU":
            self.layer = GRU(self.encoder_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        else:
            self.layer = SimpleRNN(self.encoder_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')


    def call(self, x, hidden):
        x = self.embedding(x)
        if self.layer_type == "LSTM":
            output, h, c = self.layer(x, initial_state = hidden)
            return output, h, c
        else:
            output, h = self.layer(x, initial_state = hidden)
            return output, h, None

    def initialize_hidden_state(self):
        if self.layer_type == "LSTM":
            return [tf.zeros((self.batch_size, self.encoder_units)), tf.zeros((self.batch_size, self.encoder_units))]
        else:
            return tf.zeros((self.batch_size, self.encoder_units))

#### Test Encoder Stack

In [41]:
## Test Encoder Stack

encoder = Encoder(num_encoder_tokens, embedding_dim, units, dataset.batch_size, "SimpleRNN")


# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h.shape))
if encoder.layer_type == "LSTM":
    print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c.shape))

Encoder output shape: (batch size, sequence length, units) (32, 22, 128)
Encoder h vecotr shape: (batch size, units) (32, 128)


## Decoder

In [35]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, decoder_units, batch_size, layer_type, attention_type='luong'):
        super(Decoder, self).__init__()
        
        self.decoder_units = decoder_units
        self.layer_type = layer_type
        self.attention_type = attention_type
        self.batch_size = batch_size
        
        # Embedding Layer
        self.embedding = Embedding(vocab_size, embedding_dim)
        
        # Final Dense layer on which softmax will be applied
        self.fc = Dense(vocab_size)
        
        # fundamental cell for decoder recurrent structure
        if self.layer_type == "LSTM":
            self.decoder_rnn_cell = LSTMCell(self.decoder_units)
        elif self.layer_type == "GRU":
            self.decoder_rnn_cell = GRUCell(self.decoder_units)
        else:
            self.decoder_rnn_cell = SimpleRNNCell(self.decoder_units)

        # Sampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()

        # Create attention mechanism with memory = None
        self.attention = self.build_attention_mechanism(self.decoder_units, 
                                                        None, self.batch_size*[max_decoder_seq_length], 
                                                        self.attention_type)

        # Wrap attention mechanism with the fundamental rnn cell of decoder
        self.rnn_cell = self.build_rnn_cell()

        # Define the decoder with respect to fundamental rnn cell
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)

        
    def build_rnn_cell(self):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell,
                                               self.attention,
                                               attention_layer_size = self.decoder_units)
        return rnn_cell
    
    def build_attention_mechanism(self, decoder_units, memory, 
                                  memory_sequence_length, attention_type='luong'):
        if(attention_type=='bahdanau'):
            return tfa.seq2seq.BahdanauAttention(units=decoder_units, memory = memory, memory_sequence_length = memory_sequence_length)
        else:
            return tfa.seq2seq.LuongAttention(units=decoder_units, memory = memory, memory_sequence_length = memory_sequence_length)

    def build_initial_state(self, batch_size, encoder_state, Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size = self.batch_size, dtype = Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state = encoder_state)
        return decoder_initial_state
    
    def call(self, inputs, initial_state):
        x = self.embedding(inputs)
        outputs, _, _ = self.decoder(x, initial_state = initial_state, sequence_length = self.batch_size*[max_decoder_seq_length-1])
        return outputs

#### Test decoder stack

In [42]:
# Test decoder stack

decoder = Decoder(num_decoder_tokens, embedding_dim, units, dataset.batch_size, "SimpleRNN", 'luong')
sample_x = tf.random.uniform((dataset.batch_size, max_decoder_seq_length))
decoder.attention.setup_memory(sample_output)
if decoder.layer_type == "LSTM":
    initial_state = decoder.build_initial_state(dataset.batch_size, [sample_h, sample_c], tf.float32)
else:
    initial_state = decoder.build_initial_state(dataset.batch_size, sample_h, tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)

print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)

Decoder Outputs Shape:  (32, 20, 67)


#### Optimizer

In [43]:
optimizer = tf.keras.optimizers.Nadam()

#### Loss Function

In [45]:
def loss_function(real, pred):
    # real shape = (BATCH_SIZE, max_length_output)
    # pred shape = (BATCH_SIZE, max_length_output, tar_vocab_size )
    cross_entropy = SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    loss = cross_entropy(y_true=real, y_pred=pred)
    mask = tf.logical_not(tf.math.equal(real,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)  
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss  

#### Checkpoints (Object-based saving)

In [46]:
checkpoint_dir = './new_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)